# Vanguard ETF Analyzer (Polygon.io Only)
This notebook loops through 11 Vanguard ETFs from 2023 to 2024 using Polygon.io data, calculates indicators and risk metrics, and saves each dataset to the ETFs folder.


In [7]:
import os
import time
import requests
import numpy as np
import pandas as pd
import pandas_ta as ta
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("POLYGON_API_KEY")
os.makedirs("ETFs", exist_ok=True)

In [8]:
def calculate_risk_metrics(df):
    returns = df['close'].pct_change().dropna()
    volatility = returns.std()
    sharpe_ratio = returns.mean() / volatility if volatility != 0 else 0
    cumulative = (1 + returns).cumprod()
    peak = cumulative.cummax()
    drawdown = (cumulative - peak) / peak
    max_drawdown = drawdown.min()
    return volatility, sharpe_ratio, max_drawdown


In [9]:
required_etfs = {
    "VOO":  "US Equity - Large Cap",
    "VSGX": "International Equity - Emerging",
    "ESGV": "ESG - US Equity",
    "VDE":  "Sector - Energy",
    "VEU":  "International Equity - Developed & Emerging",
    "VHT":  "Sector - Healthcare",
    "VOOG": "US Equity - Growth",
    "VGT":  "Sector - Technology",
    "VOX":  "Sector - Communication",
    "VUG":  "US Equity - Growth",
    "VFMO": "US Equity - Momentum"
}


## Step 1: Pull OHLCV Data from Polygon.io
Checks for a cached CSV first. If it doesn't exist, fetches from the API.


In [10]:
start_date = "2023-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")
all_dfs = []
processed_etfs = set()

for symbol, asset_class in required_etfs.items():
    print(f"\n--- Processing {symbol} ({asset_class}) ---")

    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{start_date}/{end_date}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}"
    response = requests.get(url)

    if response.status_code == 429:
        print("Rate limit hit. Waiting 60 seconds...")
        time.sleep(60)
        response = requests.get(url)

    time.sleep(13)
    data = response.json().get('results', [])
    if not data:
        print(f"No data returned for {symbol}")
        continue

    df = pd.DataFrame(data)
    df['t'] = pd.to_datetime(df['t'], unit='ms')
    df.rename(columns={'t': 'date', 'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume'}, inplace=True)
    df = df[['date', 'open', 'high', 'low', 'close', 'volume']]
    df.set_index('date', inplace=True)

    # Technical indicators
    df['rsi'] = ta.rsi(df['close'], length=14)
    df['macd'] = ta.macd(df['close'])['MACD_12_26_9']
    df['sma50'] = ta.sma(df['close'], length=50)
    df['ema20'] = ta.ema(df['close'], length=20)
    df['atr'] = ta.atr(df['high'], df['low'], df['close'])

    # Target and dropna
    df['next_day_return'] = df['close'].pct_change().shift(-1)
    df['target'] = (df['next_day_return'] > 0).astype(int)
    df.dropna(subset=['rsi', 'macd', 'sma50', 'ema20', 'atr', 'next_day_return'], inplace=True)

    # Risk metrics
    volatility, sharpe, max_dd = calculate_risk_metrics(df)
    df['volatility'] = volatility
    df['sharpe_ratio'] = sharpe
    df['max_drawdown'] = max_dd

    # Simulated sentiment
    np.random.seed(42)
    df['daily_sentiment'] = np.random.normal(loc=0.02, scale=0.1, size=len(df))

    # Metadata
    df['symbol'] = symbol
    df['asset_class'] = asset_class

    # Save to CSV
    df.to_csv(f"ETFs/{symbol}_enriched.csv")
    all_dfs.append(df)
    processed_etfs.add(symbol)
    print(f"{symbol} range: {df.index.min().date()} to {df.index.max().date()}")


--- Processing VOO (US Equity - Large Cap) ---
VOO range: 2023-06-20 to 2025-04-03

--- Processing VSGX (International Equity - Emerging) ---
VSGX range: 2023-06-20 to 2025-04-03

--- Processing ESGV (ESG - US Equity) ---
ESGV range: 2023-06-20 to 2025-04-03

--- Processing VDE (Sector - Energy) ---
VDE range: 2023-06-20 to 2025-04-03

--- Processing VEU (International Equity - Developed & Emerging) ---
VEU range: 2023-06-20 to 2025-04-03

--- Processing VHT (Sector - Healthcare) ---
VHT range: 2023-06-20 to 2025-04-03

--- Processing VOOG (US Equity - Growth) ---
VOOG range: 2023-06-20 to 2025-04-03

--- Processing VGT (Sector - Technology) ---
VGT range: 2023-06-20 to 2025-04-03

--- Processing VOX (Sector - Communication) ---
VOX range: 2023-06-20 to 2025-04-03

--- Processing VUG (US Equity - Growth) ---
VUG range: 2023-06-20 to 2025-04-03

--- Processing VFMO (US Equity - Momentum) ---
VFMO range: 2023-06-20 to 2025-04-03


In [12]:
# Auto-discover additional Vanguard ETFs not already processed
search_url = f"https://api.polygon.io/v3/reference/tickers?market=stocks&type=ETF&active=true&limit=1000&apiKey={api_key}"
response = requests.get(search_url).json()

auto_discovered = {
    r["ticker"]: r["name"]
    for r in response.get("results", [])
    if "Vanguard" in r.get("name", "") and r["ticker"] not in required_etfs
}

print(f"\nFound {len(auto_discovered)} additional Vanguard ETFs:")
print(list(auto_discovered.keys()))

for symbol, name in auto_discovered.items():
    print(f"\n--- Auto-processing {symbol} ({name}) ---")

    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/1/day/{start_date}/{end_date}?adjusted=true&sort=asc&limit=50000&apiKey={api_key}"
    response = requests.get(url)

    if response.status_code == 429:
        print("Rate limit hit. Waiting 60 seconds...")
        time.sleep(60)
        response = requests.get(url)

    time.sleep(13)

    data = response.json().get('results', [])
    if not data:
        print(f"No data returned for {symbol}")
        continue

    df = pd.DataFrame(data)
    df['t'] = pd.to_datetime(df['t'], unit='ms')
    df.rename(columns={'t': 'date', 'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close', 'v': 'volume'}, inplace=True)
    df = df[['date', 'open', 'high', 'low', 'close', 'volume']]
    df.set_index('date', inplace=True)

    df['rsi'] = ta.rsi(df['close'], length=14)
    df['macd'] = ta.macd(df['close'])['MACD_12_26_9']
    df['sma50'] = ta.sma(df['close'], length=50)
    df['ema20'] = ta.ema(df['close'], length=20)
    df['atr'] = ta.atr(df['high'], df['low'], df['close'])

    df['next_day_return'] = df['close'].pct_change().shift(-1)
    df['target'] = (df['next_day_return'] > 0).astype(int)
    df.dropna(subset=['rsi', 'macd', 'sma50', 'ema20', 'atr', 'next_day_return'], inplace=True)

    volatility, sharpe, max_dd = calculate_risk_metrics(df)
    df['volatility'] = volatility
    df['sharpe_ratio'] = sharpe
    df['max_drawdown'] = max_dd

    np.random.seed(42)
    df['daily_sentiment'] = np.random.normal(loc=0.02, scale=0.1, size=len(df))

    df['symbol'] = symbol
    df['asset_class'] = "Auto-Discovered"

    df.to_csv(f"ETFs/{symbol}_enriched.csv")
    all_dfs.append(df)

    print(f"{symbol} range: {df.index.min().date()} to {df.index.max().date()}")



Found 7 additional Vanguard ETFs:
['BIV', 'BLV', 'BND', 'BNDW', 'BNDX', 'BSV', 'EDV']

--- Auto-processing BIV (Vanguard Intermediate-Term Bond ETF) ---
BIV range: 2023-06-20 to 2025-04-03

--- Auto-processing BLV (Vanguard Long-Term Bond ETF) ---
BLV range: 2023-06-20 to 2025-04-03

--- Auto-processing BND (Vanguard Total Bond Market) ---
BND range: 2023-06-20 to 2025-04-03

--- Auto-processing BNDW (Vanguard Total World Bond ETF) ---
BNDW range: 2023-06-20 to 2025-04-03

--- Auto-processing BNDX (Vanguard Total International Bond ETF) ---
BNDX range: 2023-06-20 to 2025-04-03

--- Auto-processing BSV (Vanguard Short-Term Bond ETF) ---
BSV range: 2023-06-20 to 2025-04-03

--- Auto-processing EDV (Vanguard World Funds Extended Duration ETF) ---
EDV range: 2023-06-20 to 2025-04-03


In [11]:
etf_master_df = pd.concat(all_dfs)
etf_master_df.to_csv("ETFs/vanguard_etfs_combined1.2.2.csv")
print("Saved combined dataset.")
etf_master_df.head()

Saved combined dataset.


,open,high,low,close,volume,rsi,macd,sma50,ema20,atr,next_day_return,target,volatility,sharpe_ratio,max_drawdown,daily_sentiment,symbol,asset_class
date,,,,,,,,,,,,,,,,,,
2023-06-20 04:00:00,403.49,404.32,401.26,403.38,4246881.0,69.667455,6.187024,383.9880,394.373661,4.104436,-0.005479,0,0.008412,0.057968,-0.123317,0.069671,VOO,US Equity - Large Cap
2023-06-21 04:00:00,402.31,403.06,400.67,401.17,3423543.0,64.792494,5.953397,384.4842,395.020931,4.002323,0.003639,1,0.008412,0.057968,-0.123317,0.006174,VOO,US Equity - Large Cap
2023-06-22 04:00:00,400.28,402.72,399.95,402.63,3141021.0,66.462130,5.818978,385.0086,395.745604,3.912243,-0.007501,0,0.008412,0.057968,-0.123317,0.084769,VOO,US Equity - Large Cap
2023-06-23 04:00:00,399.33,401.28,398.93,399.61,4215307.0,60.111947,5.406439,385.5026,396.113642,3.896754,-0.004129,0,0.008412,0.057968,-0.123317,0.172303,VOO,US Equity - Large Cap
2023-06-26 04:00:00,399.04,400.86,397.73,397.96,3250700.0,56.912454,4.889989,385.8664,396.289486,3.840886,0.011082,1,0.008412,0.057968,-0.123317,-0.003415,VOO,US Equity - Large Cap
